In [36]:

from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from common import (
    format_ax,
    query_mssql_iteratively,
    create_emission_source_query,
    PASSWORD
)
from config import CUSTOMER, FINAL_REPORTS_PATH, FINAL_REPORT_COLUMN, REPORTS_AND_MONTHS

print(CUSTOMER)


italgas


In [11]:
def create_survey_query_extended(survey_id):
    query = f"""
    SELECT CONVERT(NVARCHAR(50), Id) AS SurveyId,
        CONVERT(NVARCHAR(50), AnalyzerId) AS AnalyzerId,
        StartEpoch,
        EndEpoch,
        EndDateTime   AS EndDateTimeSurvey,
        DrivingLengthMeters / 1000 AS DrivingLengthKM
    FROM Survey
    WHERE Id = '{survey_id}'      
    """
    return query

In [2]:
### Load the reports
report_prefixes_with_date = pd.read_excel(
    FINAL_REPORTS_PATH,
)[REPORTS_AND_MONTHS]

assert (
    len(report_prefixes_with_date[FINAL_REPORT_COLUMN])
    == report_prefixes_with_date[FINAL_REPORT_COLUMN].nunique()
)
print(len(report_prefixes_with_date[FINAL_REPORT_COLUMN]))

1153


/home/ubuntu/miniconda3/envs/tensorflow-analytics/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [23]:
report_prefixes_with_date_to_download = report_prefixes_with_date["Finalreportpcubed"]
report_prefixes_with_date_to_download  = report_prefixes_with_date_to_download.str.split("-").map(lambda x: x[-1]
)

In [24]:
report_prefixes_with_date_to_download_list = list(report_prefixes_with_date_to_download)


In [25]:
report_prefixes_with_date_to_download_list[0:2]

['D29DDB', '1DE0C5']

In [28]:
emission_sources = query_mssql_iteratively(
    query_parameters=report_prefixes_with_date_to_download_list,
    query_function=create_emission_source_query,
)

In [29]:
### Get the survey_ids
survey_ids = emission_sources["SurveyId"].drop_duplicates()
surveys = query_mssql_iteratively(survey_ids, create_survey_query_extended)

In [33]:
surveys['Month'] = pd.to_datetime(surveys['EndDateTimeSurvey']).dt.month
surveys['Year'] = 2022

In [38]:
# Group by month and year, then calculate sum
grouped = surveys.groupby(['Year', 'Month']).agg({'DrivingLengthKM':'sum'}).reset_index()
grouped

,Year,Month,DrivingLengthKM
0,2022,1,35042.85578
1,2022,2,47921.43955
2,2022,3,53417.63158
3,2022,4,43123.92069
4,2022,5,43704.76993
5,2022,6,44284.33564
6,2022,7,42104.09209
7,2022,8,22273.23971
8,2022,9,20434.87754
9,2022,10,22231.35572


In [37]:
# Plotting DrivingLengthKM
plt.figure(figsize=(12,6))
sns.barplot(data=grouped, x='Month', y='value', hue='Year')
plt.title('Monthly Sum of Driving Length (KM)')
plt.xlabel('Month')
plt.ylabel('Sum of Driving Length (KM)')

plt.show()

ValueError: Could not interpret input 'value'

<Figure size 1200x600 with 0 Axes>